## Introduction
This script detects gestural alignment in the Fribble tasks used in Rasenberg et al. (2022) and Akamine et al. (in preparation). In these studies, gestural alignment has been defined as "cross-participant repetition of iconic gestures referring to the same subpart of an Fribble". For example, if speaker A produced a triangle gesture to describe Fribble 02A and speaker B produces a similar triangle gesture to describe Fribble 02A, we call it a gestural alignment. However, if speaker B produced a similar triangle gesture to describle different subparts, then it is not a gestural alignment.

We coded references of each iconic gesture in ELAN, and the exported tsv files will be used in this script.

## Load tsv files and preprocess the data

In [1]:
### Import libraries
import os
# %load_ext cudf.pandas  # pandas operations now use the GPU!
import pandas as pd
import numpy as np
import itertools
# import pympi
from tqdm import tqdm


### Define folder paths
original_elan_folder = f"01_elan_output/"
processed_elan_folder = f"02_elan_output_processed/"
referent_df_folder = f"03_referent_df/"
gestural_alignment_folder = "04_gestural_alignment/"

In [33]:
if not os.path.exists(os.path.join(processed_elan_folder, 'elan_annotation_merged.txt')):    
    df_annot_gesture = pd.read_csv(os.path.join(original_elan_folder, 'elan_annotation_gesture_only.txt'), 
                                sep='\t', skip_blank_lines=True)
    df_annot_all = pd.read_csv(os.path.join(original_elan_folder, 'elan_annotation_all.txt'), 
                                sep='\t', skip_blank_lines=True, index_col=False, dtype={'trial': str})

    # change the format of trial column from 1.1 to "1_1"
    df_annot_all['trial'] = df_annot_all['trial'].str.replace('.', '_')

    ### merge the two dataframes
    merge_on = ["Begin Time - msec", "File"] # _gesture tiers shouldn't be used as merge_on because full annotation may have different values due to temporal co-occurances
    # drop _gesture columns and time columns to avoid having multiple columns with the same name (e.g. A_LH_gesture_x, A_LH_gesture_y)
    df_annot_all = df_annot_all.drop(columns=['A_LH_gesture', 'A_RH_gesture', 'B_LH_gesture', 'B_RH_gesture', 
                                                'End Time - msec', 'Duration - msec'])
    df_merged_annot = pd.merge(df_annot_gesture, df_annot_all, on=merge_on, how='left')

    # check if the number of rows is the same
    print(len(df_merged_annot))
    print(len(df_annot_gesture))

    #export the merged dataframe
    df_merged_annot.to_csv(os.path.join(processed_elan_folder, 'elan_annotation_merged.txt'), sep='\t', index=False)

C:\Users\shoaka\AppData\Local\Temp\ipykernel_4700\1775675640.py:4: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_annot_all = pd.read_csv(os.path.join(original_elan_folder, 'elan_annotation_all.txt'),
C:\Users\shoaka\AppData\Local\Temp\ipykernel_4700\1775675640.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_annot_all['trial'] = df_annot_all['trial'].str.replace('.', '_')


14271
14271


In [2]:
# Read a file, preprocess it, and return it as a pandas dataframe
def read_file(file):
    input_file = os.path.join(processed_elan_folder, file)
    df = pd.read_csv(input_file, index_col=False, sep='\t')
    df['pair'] = df['File'].str.split('.eaf').str[0]
    df = select_gesture_only(df)
    export_representational_gesture(df)
    df = select_iconic_gesture(df)
    df.to_csv(processed_elan_folder + "all_iconic_gestures.csv", index=False)
    df = gesture_referent(df)
    df = select_columns(df)
    df = df.reset_index(drop=True)
    return df


# def read_eaf(file):
#     input_file = os.path.join(elan_folder, file)
#     eafob = pympi.Elan.Eaf(input_file)
#     tiers = eafob.get_tier_names()
#     # make sure you set the name of the tier where the annotations of interest are located
#     annots = eafob.get_annotation_data_for_tier(tiers)
#     return annots


def select_gesture_only(df):
    # Select only rows where the gesture columns are not empty or non-gesture
    df = df.loc[(df['A_LH_gesture'] == "gesture") | (df['A_RH_gesture'] == "gesture") | (df['B_LH_gesture'] == "gesture") | (df['B_RH_gesture'] == "gesture")]
    print('Number of gestures:', len(df))
    # Create a new column that indicates the gesturer for each row
    conditions = [
        df['A_LH_gesture'] == "gesture",
        df['A_RH_gesture'] == "gesture",
        df['B_LH_gesture'] == "gesture",
        df['B_RH_gesture'] == "gesture"
    ]
    choices = ['A', 'A', 'B', 'B']
    df.loc[:, 'gesturer'] = np.select(conditions, choices, default=np.nan)
    return df


def export_representational_gesture(df):
    '''
    This function selects representational gestures (iconic + deictic) from the dataframe.
    It creates a new column 'representational' that is True if the gesture is representational and False otherwise.

    This will be used to compare the frequency of representational gestures between the two datasets.
    We don't compare freq of iconic gestures, because some gestures were operationalized as deictic in the zoom dataset were coded as iconic in the cabb_small dataset.
    In particular, loose-hand gestures that only depict the location of referents were coded as deictic in the zoom dataset, but as iconic in the cabb_small dataset.
    '''
    df = df.copy()

    # speaker A
    df['iconic_a'] = df.apply(lambda row: True if (row['A_LH_gesture_iconicity'] == 1 or row['A_RH_gesture_iconicity'] == 1) else False, axis=1)
    df['deictic_a'] = df.apply(lambda row: True if (row['A_LH_gesture_deixis'] == 1 or row['A_RH_gesture_deixis'] == 1) else False, axis=1)
    # speaker B
    df['iconic_b'] = df.apply(lambda row: True if (row['B_LH_gesture_iconicity'] == 1 or row['B_RH_gesture_iconicity'] == 1) else False, axis=1)
    df['deictic_b'] = df.apply(lambda row: True if (row['B_LH_gesture_deixis'] == 1 or row['B_RH_gesture_deixis'] == 1) else False, axis=1)
    # combined
    df['iconic'] = np.where(((df['gesturer'] == 'A') & (df['iconic_a'] == True)) |
                            ((df['gesturer'] == 'B') & (df['iconic_b'] == True)), True, False)
    df['deictic'] = np.where(((df['gesturer'] == 'A') & (df['deictic_a'] == True)) |
                            ((df['gesturer'] == 'B') & (df['deictic_b'] == True)), True, False)
    df['representational'] = np.where(df['iconic'] | df['deictic'], True, False)

    # change true and false to 1 and 0
    df['representational'] = df['representational'].astype(int)
    df['iconic'] = df['iconic'].astype(int)
    df['deictic'] = df['deictic'].astype(int)

    # change columns names for time
    df = df.rename(columns={'Begin Time - msec': 'begin_time', 'End Time - msec': 'end_time', 'Duration - msec': 'duration'})

    # select columns
    columns = ['pair', 'round', 'trial', 'gesturer', 'target', 'accuracy',
                'begin_time', 'end_time', 'duration',
                'iconic', 'deictic', 'representational']
    df = df[columns]

    # export representational gestures to a csv file
    df.to_csv(processed_elan_folder + 'gestures_type.csv', index=False)


# Select only iconic gestures from the dataframe
# Gesture type columns end with _gesture_type, and iconic gestures are labeled as 'iconic'
# If all of the gesture type columns is not labeled as 'iconic', the row is dropped
def select_iconic_gesture(df):      
    iconic_columns = [col for col in df.columns if col.endswith('iconicity')]
    iconic_columns_a = [col for col in iconic_columns if col.startswith('A')]
    iconic_columns_b = [col for col in iconic_columns if col.startswith('B')]
    df['iconic_a'] = df[iconic_columns_a].apply(lambda x: any(x.values == 1), axis=1)
    df['iconic_b'] = df[iconic_columns_b].apply(lambda x: any(x.values == 1), axis=1)

    # set iconic column to True if any of the iconic_a or iconic_b columns is True and if the gesturer for the row is the same as the iconic gesture
    df['iconic'] = np.where(((df['iconic_a'] == True) & (df['gesturer'] == 'A')) |
                            ((df['iconic_b'] == True) & (df['gesturer'] == 'B')), True, False)
    df = df.loc[df['iconic'] == True]
    print('Number of iconic gestures:', len(df))
    return df

# Create two new columns that contain the gesture's referent for each speaker A and B
# The referent is coded in the columns that end with _gesture_referent
# The first column of each speaker is filled with the referent of right hand. If the referent of right hand is empty, the referent of left hand is filled
# The second column of each speaker is filled with the referent of left hand only if the referents of right is NOT empty and referents of left and right hand gestures are diferent
# If the referents of left and right hand gestures are the same, the second column of each speaker is left blank
def gesture_referent(df):
    # Create the first referent columns for each speaker and fill them with the referent of right hand
    df['A_gesture_referent'] = df['A_RH_gesture_referent']
    df['B_gesture_referent'] = df['B_RH_gesture_referent']
    
    # Fill the columns with the referent of left hand if the referent of right hand is empty
    df.loc[df['A_gesture_referent'].isnull(), 'A_gesture_referent'] = df['A_LH_gesture_referent']
    df.loc[df['B_gesture_referent'].isnull(), 'B_gesture_referent'] = df['B_LH_gesture_referent']

    # Create the second referent columns for each speaker and fill them with the referent of left hand only 
    # if the referents of right is NOT empty and referents of left and right hand gestures are diferent
    df.loc[(df['A_gesture_referent'].notnull()) & (df['A_LH_gesture_referent'] != df['A_gesture_referent']), 'A_gesture_referent2'] = df['A_LH_gesture_referent']
    df.loc[(df['B_gesture_referent'].notnull()) & (df['B_LH_gesture_referent'] != df['B_gesture_referent']), 'B_gesture_referent2'] = df['B_LH_gesture_referent']

    # remove -REP from the referent columns
    df['A_gesture_referent'] = df['A_gesture_referent'].str.replace('-REP', '')
    df['B_gesture_referent'] = df['B_gesture_referent'].str.replace('-REP', '')
    df['A_gesture_referent2'] = df['A_gesture_referent2'].str.replace('-REP', '')
    df['B_gesture_referent2'] = df['B_gesture_referent2'].str.replace('-REP', '')

    # remove rows if the referent contains "-rep", 'nan', 'main', 'undecided'; ~ means not
    print('Number of rows before removing -rep, nan, main, general, undecided:', len(df))
    df = df.loc[~df['A_gesture_referent'].str.contains('-rep|nan|main|undecided', na=False)]
    df = df.loc[~df['B_gesture_referent'].str.contains('-rep|nan|main|undecided', na=False)]
    df = df.loc[~df['A_gesture_referent2'].str.contains('-rep|nan|main|undecided', na=False)]
    df = df.loc[~df['B_gesture_referent2'].str.contains('-rep|nan|main|undecided', na=False)]
    print('Number of rows after removing -rep, nan, main, general, undecided:', len(df))

    # for "other" referents, add target (float type) at the beginning (e.g., "other" -> "09_other", "other" -> "03_other")
    # we can convert target to string and concatenate it with the referent if the referent is "other"
    df['A_gesture_referent'] = df.apply(lambda row: f"{int(row['target']):02d}_{row['A_gesture_referent']}" 
                                        if row['A_gesture_referent'] == 'other' 
                                        else row['A_gesture_referent'], axis=1)
    df['B_gesture_referent'] = df.apply(lambda row: f"{int(row['target']):02d}_{row['B_gesture_referent']}" 
                                        if row['B_gesture_referent'] == 'other' 
                                        else row['B_gesture_referent'], axis=1)
    df['A_gesture_referent2'] = df.apply(lambda row: f"{int(row['target']):02d}_{row['A_gesture_referent2']}" 
                                        if row['A_gesture_referent2'] == 'other' 
                                        else row['A_gesture_referent2'], axis=1)
    df['B_gesture_referent2'] = df.apply(lambda row: f"{int(row['target']):02d}_{row['B_gesture_referent2']}" 
                                        if row['B_gesture_referent2'] == 'other' 
                                        else row['B_gesture_referent2'], axis=1)
    return df


# Select columns that will be used for the analysis
def select_columns(df):
    columns = ['pair', 'round', 'trial', 'director', 'target', 'accuracy', 'A_speech', 'B_speech', 'A_speech_eng', 'B_speech_eng',
               'A_LH_gesture_referent', 'A_RH_gesture_referent', 'B_LH_gesture_referent', 'B_RH_gesture_referent', 
               'A_gesture_referent', 'A_gesture_referent2', 'B_gesture_referent',  'B_gesture_referent2',
               'Begin Time - msec', 'End Time - msec', 'Duration - msec', 'iconic']
    df = df[columns]
    df = df.rename(columns={'Begin Time - msec': 'begin_time', 'End Time - msec': 'end_time', 'Duration - msec': 'duration'})
    return df


In [3]:
for filename in os.listdir(processed_elan_folder):
    if filename.endswith("_merged.txt"): 
        if os.path.exists(os.path.join(processed_elan_folder, filename.split('_merged.txt')[0] + '_clean.csv')):
            print(filename.split('_merged.txt')[0] + '_clean.csv' + ' already exists in the output folder. Skipping this file.')
        else:
            print('Processing file: ' + filename)
            clean_filename = filename.split('_merged.txt')[0] + '_clean.csv'
            clean_df = read_file(filename)
            clean_df.to_csv(processed_elan_folder + clean_filename, index=False)

Processing file: elan_annotation_merged.txt
Number of gestures: 11438


C:\Users\shoaka\AppData\Local\Temp\ipykernel_17588\2676333107.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'gesturer'] = np.select(conditions, choices, default=np.nan)


Number of iconic gestures: 7758
Number of rows before removing -rep, nan, main, general, undecided: 7758
Number of rows after removing -rep, nan, main, general, undecided: 6965


## Detect gestural alignment

To detect gestural alignment per Fribble subpart, we will take the following steps:

1. extract all the pairs and save them as a list "pairs"
2. interate the df for each pair
    1. extract all the referents used by the speakers and save them as a list "referents"
    2. iterate the df for each referent
        1. if both speakers produced iconic gestures with same referents, save the necessary information (e.g., round, trial, director, target, accuracy, speaker for each gesture) as a df

At the end of all the loops, we will have the df "gestural_alignment" which contains the necessary information for all referents and pairs.

In [35]:
### Functions
def make_alignment1_df(row):
    df = row.to_frame().transpose() #this is to make sure that the dataframe is transposed so that the dataframe is added as a row
    df = df.drop(columns=['pair', 'flag'])
    df = df.add_suffix('_1')
    df = df.reset_index(drop=True) #this is to make sure that the index of the dataframe is reset to 0; the index of two dataframes that will be concatenated should be the same
    return df

def make_alignment2_df(row):
    df = row.to_frame().transpose()
    df = df.drop(columns=['pair', 'flag'])
    df = df.add_suffix('_2')
    df = df.reset_index(drop=True)
    return df

def append_to_alignment_df(main_df, df1, df2, pair, referent):
    # concatenate the two dataframes as one row
    df = pd.concat([df1, df2], axis=1) #axis=1 is to concatenate the two dataframes as columns and make it one row
    df['pair'] = pair
    df['referent'] = referent
    main_df = pd.concat([main_df, df], ignore_index=True)
    main_df = main_df.reset_index(drop=True)
    return main_df


In [38]:
#if separate_subparts = true, 14A+14B will be treated as both as 14A & 14B separately (14A as referent1 and 14B as referent2)
#for example, if 14A+14B is produced by speaker A and speaker B produced 14A+14B, this is two gestural alignment (14A & 14B)
#also, if 14A+14B is produced by speaker A and speaker B produced 14A and 14B separately, this is two gestural alignments
#likewise, if 14A is produced by speaker A and speaker B produced 14A+14B, this is one gestural alignment
#however, if speaker A produced 14A and 14B separately and speaker B produced 14A+14B, this is two gestural alignments
#for Zoom data, separate_subparts should be set to False because the current coding scheme codes the referent for each hand separately unlike Rasenberg et al. (2022)
separate_subparts = False
filename = 'elan_annotation_clean.csv'

if os.path.exists(gestural_alignment_folder + 'gestural_alignment.csv'):
    print('gestural_alignment.csv already exists in the output folder. Skipping this file.')
else:
    clean_df = pd.read_csv(processed_elan_folder + filename, dtype={'pair':str})
    clean_df.drop(columns=['A_speech', 'B_speech', 'A_speech_eng', 'B_speech_eng', 'iconic'])

    print('Number of rows in the dataframe before removing -WF: ' + str(len(clean_df)))
    # remove rows where referent columns contains -WF
    clean_df = clean_df.loc[(clean_df['A_gesture_referent'].str.contains('-WF', na=False, regex=False) == False) &
                            (clean_df['B_gesture_referent'].str.contains('-WF', na=False, regex=False) == False) &
                            (clean_df['A_gesture_referent2'].str.contains('-WF', na=False, regex=False) == False) &
                            (clean_df['B_gesture_referent2'].str.contains('-WF', na=False, regex=False) == False)]
    print('Number of rows in the dataframe after removing -WF: ' + str(len(clean_df)))

    pairs = clean_df['pair'].unique()
    referent_cols = ['A_gesture_referent', 'B_gesture_referent', 'A_gesture_referent2', 'B_gesture_referent2']

    # make an empty pandas dataframe that will contain the alignment information
    # the alignment dataframe will have the the following columns:
    alignment_df = pd.DataFrame(columns=['pair', 'referent', 'speaker_1', 'round_1', 'trial_1', 'director_1', 'target_1', 'accuracy_1', 
                                        'A_gesture_referent_1', 'B_gesture_referent_1', 'A_gesture_referent2_1', 'B_gesture_referent2_1', 
                                        'begin_time_1', 'end_time_1', 'duration_1',
                                        'speaker_2', 'round_2', 'trial_2', 'director_2', 'target_2', 'accuracy_2', 
                                        'A_gesture_referent_2', 'B_gesture_referent_2', 'A_gesture_referent2_2', 'B_gesture_referent2_2', 
                                        'begin_time_2', 'end_time_2', 'duration_2'])

    for pair in pairs:
        pair_df = clean_df.loc[clean_df['pair'] == pair]
        referents = pair_df[referent_cols].to_numpy().flatten()
        # remove nan values and duplicates
        referents = [referent for referent in referents if str(referent) not in ['nan', 'main', 'general', 'undecided']]
        
        if separate_subparts:
            # remove elements with '+' in them
            referents = [referent for referent in referents if '+' not in str(referent)]

        referents = list(dict.fromkeys(referents))
        
        for referent in referents:
            if separate_subparts:
                referent_df = pair_df.loc[(pair_df["A_gesture_referent"].str.contains(referent, na=False, regex=False)) | 
                                            (pair_df["B_gesture_referent"].str.contains(referent, na=False, regex=False)) | 
                                            (pair_df["A_gesture_referent2"].str.contains(referent, na=False, regex=False)) | 
                                            (pair_df["B_gesture_referent2"].str.contains(referent, na=False, regex=False))]
            else:
                referent_df = pair_df.loc[pair_df[referent_cols].isin([referent]).any(axis=1)]
            
            referent_df = referent_df.sort_values(by=['begin_time']).reset_index(drop=True)

            # if the speaker of the next row is different from the speaker of the current row, the flag is set to 1
            # if the speaker of the next row is the same as the speaker of the current row or it's the last row, the flag is set to 0
            referent_df['speaker'] = np.where(referent_df['A_gesture_referent'].str.contains(referent, na=False, regex=False), 'A', 
                                                np.where(referent_df['A_gesture_referent2'].str.contains(referent, na=False, regex=False), "A", 
                                                        np.where(referent_df['B_gesture_referent'].str.contains(referent, na=False, regex=False), 'B', 
                                                                np.where(referent_df['B_gesture_referent2'].str.contains(referent, na=False, regex=False), 'B', None))))
            # remove referents if the first two letters do not match the target column (except 'other')
            # make sure to convert the two letters to float to avoid errors
            referent_df = referent_df.loc[(referent_df['A_gesture_referent'].str[:2].astype(float) == referent_df['target']) |
                                            (referent_df['A_gesture_referent2'].str[:2].astype(float) == referent_df['target']) |
                                            (referent_df['B_gesture_referent'].str[:2].astype(float) == referent_df['target']) |
                                            (referent_df['B_gesture_referent2'].str[:2].astype(float) == referent_df['target'])]
            # check if the speaker of the next row is different from the speaker of the current row
            referent_df['flag'] = np.where((referent_df['speaker'] != referent_df['speaker'].shift(-1)) & (referent_df['speaker'].shift(-1).notnull()), 1, 0)

            # export the referent_df to a csv file
            referent_df.to_csv(referent_df_folder + pair + '_' + referent + '.csv', index=False)

            # add the rows that have the flag set to 1 to alignment_df
            # the rows with the flag set to 1 are the rows where the gesture alignment initiates, and the next row is where the gesture alignment is established
            # the information of the rows with the flag set to 1 is added to the alignment_df under the columns that end with _1
            # the information of the next row is added to the alignment_df under the columns that end with _2
            alignment = False # alignment is set to True when the flag is 1: this will be used to add the information of the next row to the alignment_df

            for index, row in referent_df.iterrows():
                if alignment == True:
                    alignment2_df = make_alignment2_df(row)
                    alignment_df = append_to_alignment_df(alignment_df, alignment1_df, alignment2_df, pair, referent)
                    alignment = False
                
                if row["flag"] == 1:
                    alignment1_df = make_alignment1_df(row)
                    alignment = True
                else:
                    alignment = False


Number of rows in the dataframe before removing -WF: 6965
Number of rows in the dataframe after removing -WF: 6965


C:\Users\shoaka\AppData\Local\Temp\ipykernel_4700\3631060540.py:21: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  main_df = pd.concat([main_df, df], ignore_index=True)


## Export dataframes as csv

In [39]:
filename = "gestural_alignment.csv"
if os.path.exists(gestural_alignment_folder + filename):
    print('gestural_alignment.csv already exists in the output folder. Skipping this file.')
else:
    alignment_df = alignment_df.sort_values(by=['pair', 'begin_time_1', 'referent']).reset_index(drop=True)
    alignment_df.to_csv(gestural_alignment_folder + filename, index=False)

    print(alignment_df.shape)

(1979, 46)
